In [1]:
from torchvision import transforms
from torch.utils.data import DataLoader
import os
project_root = "/home/ubuntu/MySAM"
data_path = 'sam_data'
batch_size=1

In [2]:
import re
tar_files = []
for file in os.listdir(f"{project_root}/{data_path}"):
    print(file)
    if re.fullmatch(r'^sa_000\d{3}\.tar$', file) is not None:
        tar_files.append(file)
if len(tar_files) < 10:
    raise Exception("At least 10 directory of images should be provided!")
tar_files = tar_files[:10] # use 10 directory only
os.chdir(f"{project_root}/{data_path}")
for i, tar_file in enumerate(tar_files):#Uncompress each compressed image directory and separate images and promts
    assert(os.system(f"tar -xvf {tar_file}") == 0)
    assert(os.system(f"mkdir {tar_file[:-4]}/image_dir") == 0)
    assert(os.system(f"mkdir {tar_file[:-4]}/prompt_dir") == 0)
    assert(os.system(f"mv {tar_file[:-4]}/*.jpg {tar_file[:-4]}/image_dir") == 0)
    assert(os.system(f"mv {tar_file[:-4]}/*.json {tar_file[:-4]}/prompt_dir") == 0)
    tar_files[i] = tar_file[:-4]
tar_files

sa_000028.tar
sa_000030.tar
sa_000021.tar
sa_000029.tar
sa_000025.tar
sa_000020.tar
sa_000023.tar
rm.sh
sa_000022.tar
sa_000026.tar
sa_000024.tar
sa_000027.tar
sa_000028/
sa_000028/sa_233338.jpg
sa_000028/sa_233338.json
sa_000028/sa_224587.jpg
sa_000028/sa_224587.json
sa_000030/
sa_000030/sa_233338.jpg
sa_000030/sa_233338.json
sa_000030/sa_224587.jpg
sa_000030/sa_224587.json
sa_000021/
sa_000021/sa_233338.jpg
sa_000021/sa_233338.json
sa_000021/sa_224587.jpg
sa_000021/sa_224587.json
sa_000029/
sa_000029/sa_233338.jpg
sa_000029/sa_233338.json
sa_000029/sa_224587.jpg
sa_000029/sa_224587.json
sa_000025/
sa_000025/sa_233338.jpg
sa_000025/sa_233338.json
sa_000025/sa_224587.jpg
sa_000025/sa_224587.json
sa_000020/
sa_000020/sa_233338.jpg
sa_000020/sa_233338.json
sa_000020/sa_224587.jpg
sa_000020/sa_224587.json
sa_000023/
sa_000023/sa_233338.jpg
sa_000023/sa_233338.json
sa_000023/sa_224587.jpg
sa_000023/sa_224587.json
sa_000022/
sa_000022/sa_233338.jpg
sa_000022/sa_233338.json
sa_000022/sa_2245

['sa_000028',
 'sa_000030',
 'sa_000021',
 'sa_000029',
 'sa_000025',
 'sa_000020',
 'sa_000023',
 'sa_000022',
 'sa_000026',
 'sa_000024']

In [3]:
tar_files = tar_files[:2]

In [4]:
import cv2
def load_image(image_path, root_dir):
    #image_path = "/home/ubuntu/sam_data/sa_000020/mini_image_dir/train/sam:1.jpg"
    image = cv2.imread(''.join([root_dir, '/', image_path]))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return torch.from_numpy(image).permute(2, 0, 1)
    

In [5]:
import torch
with torch.no_grad():
    from build_sam import sam_model_registry
    teacher_model_checkpoint_path = f'{project_root}/model_checkpoint/sam_vit_h_4b8939.pth'
    teacher_model = sam_model_registry['default'](teacher_model_checkpoint_path)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    _ = teacher_model.to(device)
    for file in tar_files:
        data_dir = '/'.join([project_root, data_path, file, 'image_dir'])
        embedding_dir  = '/'.join([project_root, data_path, file, 'embedding_dir'])
        if not os.path.exists(embedding_dir):
            os.mkdir(embedding_dir)
        image_path_iter = DataLoader(os.listdir(data_dir), batch_size=batch_size, shuffle=True)
        for batch_path in image_path_iter:
            input_images = torch.stack([teacher_model.preprocess(load_image(path, data_dir).to(device)) for path in batch_path])
            #input_images = torch.stack([teacher_model.preprocess(image[0].to(device)) for image in batch_image])
            #pdb.set_trace()
            teacher_emd = teacher_model.image_encoder(input_images)
            for image_file_name, embed in zip(batch_path, teacher_emd):
                target_path = ''.join([embedding_dir, '/', image_file_name.split('.')[0], '.pth'])
                torch.save(embed, target_path)
            print("process one batch")
    

process one batch
process one batch
process one batch
process one batch
